In [1]:
import pandas as pd

In [2]:
# load csv file
file_path = './raw-results/COSC_499_Test_O_Rama_Questionnaire_RAW.csv'
df = pd.read_csv(file_path)

# remove headers
df_dropped = df.drop(index=[0, 1, 2])

# separate question blocks
num_TLX_questions = 10               # total number of questions
num_TLX_sub_questions = 4            # parts per question
TLX_end_idx = num_TLX_questions*num_TLX_sub_questions

TLX = df_dropped.iloc[:, 0:TLX_end_idx]
WUS = df_dropped.iloc[:, TLX_end_idx:(TLX_end_idx+10)]
recommend = df_dropped.iloc[:, (TLX_end_idx+10):(TLX_end_idx+11)]
free_text_q1 = df_dropped.iloc[:, (TLX_end_idx+11):(TLX_end_idx+12)]
free_text_q2 = df_dropped.iloc[:, (TLX_end_idx+12):(TLX_end_idx+13)]

In [3]:
#-------------------------------------------
# Analysis for TLX
#-------------------------------------------

# function to map 7-point likert scale to 0-100
def map_to_0_100(likert_point):
    return ((likert_point - 1) / 6) * 100

# hardcoding since this is quicker
indices_to_exclude = list(range(1, TLX_end_idx, num_TLX_sub_questions))  # for question 2: "I was successful in accomplishing what I wanted to do"
reverse_code_columns = [i for i in range(TLX.shape[1]) if i not in indices_to_exclude]

# reverse code columns and scale to 0-100
TLX_int = TLX.astype(int)
TLX_int.iloc[:, reverse_code_columns] = TLX_int.iloc[:, reverse_code_columns].apply(lambda x: 8-x)
TLX_scaled=TLX_int.iloc[:, :].map(map_to_0_100)

# calculate mean for each task
# note: for each group, we are taking the mean of the mean of each column
# This gives the correct mean since the number of elements of all groups is the same
# https://math.stackexchange.com/questions/95909/why-is-an-average-of-an-average-usually-incorrect
num_groups = len(TLX_scaled.columns) // 4
for i in range(num_groups):
    group = TLX_scaled.iloc[:, i*4:(i+1)*4]
    print("Mean TLX Score for Task {task_num}:".format(task_num=i+1), 
          group.mean().mean())

# calculate mean for all tasks
print("Mean TLX Score for All Tasks:", TLX_scaled.mean().mean())

Mean TLX Score for Task 1: 81.81818181818183
Mean TLX Score for Task 2: 82.95454545454545
Mean TLX Score for Task 3: 80.68181818181819
Mean TLX Score for Task 4: 85.98484848484848
Mean TLX Score for Task 5: 78.7878787878788
Mean TLX Score for Task 6: 82.57575757575758
Mean TLX Score for Task 7: 78.03030303030305
Mean TLX Score for Task 8: 76.51515151515153
Mean TLX Score for Task 9: 82.1969696969697
Mean TLX Score for Task 10: 82.95454545454545
Mean TLX Score for All Tasks: 81.25


In [4]:
#-------------------------------------------
# Analysis for WUS
# Guidelines: https://measuringu.com/sus/
#-------------------------------------------

WUS_int = WUS.astype(int)
odd_items = [i for i in range(WUS.shape[1]) if i % 2 == 0]      # index starts at 0
even_items = [i for i in range(WUS.shape[1]) if i % 2 != 0]

# for odd items, subtract one from the user response
WUS_int.iloc[:, odd_items] = WUS_int.iloc[:, odd_items].apply(lambda x: x-1)

# for even-numbered items, subtract the user responses from 5
WUS_int.iloc[:, even_items] = WUS_int.iloc[:, even_items].apply(lambda x: 5-x)

# ensure that all values are between 0 and 4 after scaling
assert ((WUS_int >= 0) & (WUS_int <= 4)).all().all()

# add up the converted responses for each user and multiply that total by 2.5
row_sums = WUS_int.sum(axis=1) * 2.5

# ensure that all values are between 0 and 100 after scaling
assert ((row_sums >= 0) & (row_sums <= 100)).all()

# find the average WUS score
print("Mean WUS Score:", row_sums.mean())

Mean WUS Score: 71.81818181818181


In [5]:
#-------------------------------------------
# How likely are you to recommend this website to others?
# Scale: 0 to 10
#-------------------------------------------

recommend_int = recommend.astype(int)
print("Mean Recommendation Score (all participants):", recommend_int.mean().mean())

# if we remove the two non-responses (0)
drop_nan = recommend_int[recommend_int != 0]
print("Mean Recommendation Score (drop NaN):", drop_nan.mean().mean())

Mean Recommendation Score (all participants): 7.909090909090909
Mean Recommendation Score (drop NaN): 7.909090909090909


In [6]:
free_text_q1

,FreeTextQ1
3,I enjoyed the improvements from my last demo.
4,The aesthetics and obvious buttons and features
5,NaN
6,"Your UI is consistent, your exam creation work..."
7,NaN
8,I love the overall UI.
9,Looks good
10,Everything
11,"I really like the user interface, very well pu..."
12,NaN


In [7]:
free_text_q2

,FreeTextQ2
3,The grade overriding system; its not very intu...
4,Probably more color contrast for buttons that ...
5,NaN
6,Honestly I saw nothing wrong. Login took a sec...
7,NaN
8,I think that some loading animation would be a...
9,Happy with the UI and everything over all
10,Nothing
11,"For the visibility, I would use an eyeball ico..."
12,NaN
